In [6]:
%%pyspark
#Leer Json
df = spark.read.load('abfss://datossinprocesar@dlsasgen2corporativo.dfs.core.windows.net/api/industrial/shoplogix/Pollos/Pollos_20230717.json', format='json')





StatementMeta(synswindustrial, 44, 7, Finished, Available)

In [7]:
%%pyspark
#Procesar Json para estructurarlo
from pyspark.sql.types import *
from pyspark.sql.functions import *

#Flatten matriz de estructuras
def flatten(df):
   complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
   while len(complex_fields)!=0:
      col_name=list(complex_fields.keys())[0]
      print ("Procesando :"+col_name+" Tipo : "+str(type(complex_fields[col_name])))
    
      #Si es StructType convierte todos los subelementos en columnas
      if (type(complex_fields[col_name]) == StructType):
         expanded = [col(col_name+'.'+k).alias(col_name+'_'+k) for k in [ n.name for n in  complex_fields[col_name]]]
         df=df.select("*", *expanded).drop(col_name)
    
      # si es ArrayType, agrega los elementos de matriz como filas usando la función de explode
      elif (type(complex_fields[col_name]) == ArrayType):    
         df=df.withColumn(col_name,explode_outer(col_name))
    
      # volver a calcular los campos complejos restantes en el esquema    
      complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
   return df

StatementMeta(synswindustrial, 44, 8, Finished, Available)

In [8]:
#Mostrar Dataframe
df_flatten = flatten(df)
display(df_flatten)

StatementMeta(synswindustrial, 44, 9, Finished, Available)

Procesando :result Tipo : <class 'pyspark.sql.types.StructType'>
Procesando :result_areas Tipo : <class 'pyspark.sql.types.ArrayType'>
Procesando :result_areas Tipo : <class 'pyspark.sql.types.StructType'>
Procesando :result_areas_machines Tipo : <class 'pyspark.sql.types.ArrayType'>
Procesando :result_areas_machines Tipo : <class 'pyspark.sql.types.StructType'>
Procesando :result_areas_machines_jobNames Tipo : <class 'pyspark.sql.types.ArrayType'>
Procesando :result_areas_machines_jobNames Tipo : <class 'pyspark.sql.types.StructType'>
Procesando :result_areas_machines_jobNames_filters Tipo : <class 'pyspark.sql.types.StructType'>
Procesando :result_areas_machines_jobNames_metrics Tipo : <class 'pyspark.sql.types.StructType'>


SynapseWidget(Synapse.DataFrame, e31c6f19-89b1-4b54-8e14-0f3278c9e3eb)

In [6]:
#Escribir en parquet
path_parquet = "abfss://datossemiprocesados@dlsasgen2corporativo.dfs.core.windows.net/industrial/lhdwindustrial/des/Shoplogix_full_0600_18072023.parquet" 

df.write \
    .mode("overwrite") \
    .format("parquet") \
    .save(path_parquet)